In [2]:
###################################### ESTRATEGIA BB
###################################################################################################################
import modulos as md
import numpy as np
import constantes as cons
import os
import sys

lista_filtrada = []
#lista=md.lista_de_monedas ()
#lista=md.filtradodemonedas ()
lista=['LINKUSDT',
 'LTCUSDT',
 'BCHUSDT',
 'WLDUSDT',
 'IDUSDT',
 'HBARUSDT',
 'API3USDT',
 'BNBUSDT',
 'C98USDT',
 '1000SHIBUSDT',
 'STXUSDT',
 'LPTUSDT',
 'XLMUSDT',
 'CRVUSDT',
 'MKRUSDT',
 'ALGOUSDT',
 'HIGHUSDT',
 'COMPUSDT',
 'DOGEUSDT',
 'XRPUSDT',
 'OPUSDT',
 'ARBUSDT',
 'AUDIOUSDT',
 '1000PEPEUSDT',
 'MATICUSDT',
 'APEUSDT',
 'SOLUSDT',
 'TOMOUSDT',
 'EOSUSDT',
 'ADAUSDT',
 'SUIUSDT',
 'YGGUSDT',
 'DODOXUSDT']

#lista=['AUDIOUSDT']

for symbol in lista:    
    try:
        data,porcentajeentrada = md.estrategia_santa(symbol,tp_flag = True)
        #data = md.estrategia_triangulos(symbol, tp_flag = True, print_lines_flag = True)
        #######################################################################
        #resultado = md.backtesting(data, plot_flag = True)
        resultado = md.backtestingsanta(data, plot_flag = False)
        if resultado['Return [%]'] >= 0:# and resultado['# Trades']!=0 :
            #if ((resultado['Profit Factor'] > 2 or np.isnan(resultado['Profit Factor'])) and (resultado['Return [%]']/resultado['# Trades']) >=0.33):            
                lista_filtrada.append(symbol)
        print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {resultado['Profit Factor']} - Win Rate [%]: {resultado['Win Rate [%]']}")
    except Exception as falla:
        _, _, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("\nError: "+str(falla)+" - line: "+str(exc_tb.tb_lineno)+" - file: "+str(fname)+"\n")
for symbol in lista_filtrada:
    md.printandlog(cons.nombrelog,symbol,pal=1)
#md.dibuja_patrones_triangulos (data,998)    


LINKUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
LTCUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
BCHUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
WLDUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
IDUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
HBARUSDT - Return [%]: 1.97% - # Trades: 6 - Profit Factor: nan - Win Rate [%]: 100.0

Error:  - line: 847 - file: modulos.py


Error: 'NoneType' object is not subscriptable - line: 55 - file: 1385556833.py

BNBUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan

Error:  - line: 847 - file: modulos.py


Error: 'NoneType' object is not subscriptable - line: 55 - file: 1385556833.py


Error: local variable 'data' referenced before assignment - line: 50 - file: 1385556833.py

STXUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan

Er

c:\LUCHO\personal\repopersonal\luchocrypto\FENIX\modulos.py:846: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  if plot_flag:


MKRUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
ALGOUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan

Error:  - line: 847 - file: modulos.py


Error: 'NoneType' object is not subscriptable - line: 55 - file: 1385556833.py

COMPUSDT - Return [%]: 1.05% - # Trades: 7 - Profit Factor: 2.3035247150689577 - Win Rate [%]: 71.42857142857143
DOGEUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
XRPUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
OPUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
ARBUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
AUDIOUSDT - Return [%]: -11.38% - # Trades: 25 - Profit Factor: 0.2872076878184534 - Win Rate [%]: 60.0

Error: local variable 'data' referenced before assignment - line: 50 - file: 1385556833.py

MATICUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate 

In [ ]:
md.dibuja_patrones_triangulos (data,998) 